In [ ]:
data = pd.read_csv('./full_results_for_crystalization.csv',index_col = 0)
data.head(30)

In [ ]:
uniq_entity_data = data[['pdb_id',
 'entity_id',
 'seq',
 'method',
 'details',
 'details_results',
 'method_results',
 'full_results']].drop_duplicates(subset='pdb_id', keep=False, inplace=False)

In [ ]:
test_id = set( pd.Series(uniq_entity_data.index).sample(20000) )

data.loc[test_id].to_csv('data/test.csv')

In [ ]:
uniq_id = set(uniq_entity_data.index) - test_id

data.loc[uniq_id].to_csv('data/uniq.csv')

In [ ]:
train_id = set(data.index) - test_id
data.loc[train_id].to_csv('data/train.csv')

In [ ]:
data_200k = data.loc[uniq_id]

data_200k['ag_index'] = data_200k.seq.map(lambda x: [pos for pos, char in enumerate(x) if char.lower() in 'aAgG'])

In [ ]:
translate = {'a':'g','g':'a','A':'G','G':'A'}

list_mutation_df = [data_200k] * 4

In [ ]:
def mutation(row,i):
    if len(row['ag_index']) <= i:
        return
    index = row['ag_index'][i]
    chars = list(row['seq'])
    chars[index] = translate[row['seq'][index]]
    row['seq'] = ''.join(chars)

In [ ]:
for i in tqdm(range(4)):
    list_mutation_df[i].apply( lambda x: mutation(x,i),axis = 1)

In [ ]:
data_1000k = pd.concat([data_200k] + list_mutation_df)
data_1000k = data_1000k.drop_duplicates(subset = ['pdb_id','entity_id','seq'])
data_1000k.to_csv('data/1000k.csv')

In [ ]:
data_200k = data.iloc[list_raw]